# P3 - Data Wrangling (Python, SQL)

## James Cooper

Project Details (Wrangle OpenStreetMap Data):

Location: Glasgow - I chose this city as this is where I live. It's a nice city and I'd like to contribute to the clarity of the data associated with it.

Objective:

Choose any area of the world in https://www.openstreetmap.org and use data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data for a part of the world that you care about.

References:

    Udacity "Data Wrangling with SQL"
    Udacity forums
    Stack Overflow

Contents:

    1 - High Level Overview
    2 - Problems in the data: Audit - Streetnames / Phone Number / Postcode
    3 - Cleaning the data / Importing to SQL / Creating Database
    4 - SQL analysis
    5 - Conclusion


# 1 - High Level Overview

First I will set up the file by importing the required libraries.

In [14]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
from collections import defaultdict
import sqlite3

import cerberus

import schema

We can explore the XML file to get a feel for the contents by counting the top level tags.

The following code completed as part of the Udacity problem sets:

In [7]:
#initialise empty dictionary
tags = {}
def count_tags(filename):
        # loop over elementtree object
    for event, elem in ET.iterparse(filename):
        if elem.tag in tags.keys():
            tags[elem.tag]+= 1
        else:
            tags[elem.tag] = 1
    return tags

def count():

    tags = count_tags('G129.osm')
    pprint.pprint(tags)


count()

{'bounds': 1,
 'member': 26514,
 'nd': 657766,
 'node': 577196,
 'osm': 1,
 'relation': 1061,
 'tag': 367900,
 'way': 93605}


The OSM wiki explains the basic structure of the data. In this instance we see that there are just short of 600,000 nodes in the dataset - these represent specific points on the map, and have associated attributes embedded in the node tag itself. Some nodes have children - additional information about each node is embedded in the child tags. As this is a top level search these child tags are not returned here.

Ways are lists of nodes that make up an area, route or street. Each way also has its own attributes.



### Filesizes

In [8]:
import os

print 'OSM filesize = {} MB'.format(os.path.getsize('G129.osm')/1.0e6)
print 'Sample filesize = {} MB'.format(os.path.getsize('G20.osm')/1.0e6)
print 'csv - nodes filesize = {} MB'.format(os.path.getsize('nodes.csv')/1.0e6)
print 'csv - nodes_tags filesize = {} MB'.format(os.path.getsize('nodes_tags.csv')/1.0e6)
print 'csv - ways filesize = {} MB'.format(os.path.getsize('ways.csv')/1.0e6)
print 'csv - ways_nodes filesize = {} MB'.format(os.path.getsize('ways_nodes.csv')/1.0e6)
print 'csv - ways_tags filesize = {} MB'.format(os.path.getsize('ways_tags.csv')/1.0e6)
print 'database filesize = {} MB'.format(os.path.getsize('database.db')/1.0e6)

OSM filesize = 131.786712 MB
Sample filesize = 6.665162 MB
csv - nodes filesize = 47.936789 MB
csv - nodes_tags filesize = 5.368172 MB
csv - ways filesize = 5.620808 MB
csv - ways_nodes filesize = 16.240482 MB
csv - ways_tags filesize = 8.00229 MB
database filesize = 94.425088 MB


### 2Tagtypes.py 

This section is intended to highlight any issues within the k attribute of each tag. The code was provided in the lesson.
Each key is assigned to one of four groups: lower means that the key includes lower case characters.
Lower_colon includes colons. problemchars includes any characters such as [=\+/&<>;\'"\?%#$@\,\. \t\r\n]. other covers any keys not meeting the above criteria.

In [9]:
filename = 'G129.osm'

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

#function to sort tags into categories based on K attribute
def key_type(element, keys):
    #ensures that we are only looking at the TAGS returned by the Etree parser called tags
    if element.tag == "tag":
        #ensures we are looking at the attribute 'K' within the tag for that same element
        k = element.attrib['k']
        #applies the regex function to the k tag - if it matches it resolves to TRUE
        if re.search(lower, k):
            keys["lower"] += 1
        elif re.search(lower_colon, k):
            keys["lower_colon"] += 1
        elif re.search(problemchars, k):
            keys["problemchars"] += 1
        else:
            keys["other"] += 1
        pass

    return keys
    
def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


def maptags():
    keys = process_map(filename)
    pprint.pprint(keys)

maptags()


{'lower': 318604, 'lower_colon': 26284, 'other': 23011, 'problemchars': 1}


We can see that out of the entire dataset only one of the keys includes problematic characters.


### 3Countuser.py

As part of the exercises the following funciton was written in order to return the total number of users who had made modifications to in the dataset.

In [10]:
def get_user(element):
    #if you see uid in the attributes list of the elements
    if "uid" in element.attrib:
        #this function will produce the number assigned to that attribute
        return element.attrib["uid"]


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        uid = get_user(element)
        #if there is not nothing in the UID field, add the uid to the output
        if uid != None:
            users.add(uid)
        pass

    return len(users)

def uniqueusers():

    users = process_map('G129.osm')
    pprint.pprint(users)
    
uniqueusers()

818


#  2 - Problems in the data: Audit - Streetnames / Phone Number / Postcode

# Audit.py

Purpose of this function is to return all values not matching a specific format.
The fields I am looking to clean are Streetnames, phone numbers and postcodes.

The code builds on that provided in the exercise  - I have added to the expected list for streetnames to reflect that there are several extra common words used in Glasgow to refer to streets, based on the previous instance of this code which was run on a 5% sample of the full dataset. This will ensure that they are not showing up erroneously in the Audit. 


In [11]:
osmfile = "G129.osm"

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

regex1 = re.compile(r'\+44\-\d{3}\-\d{3}\-\d{4}')

regex2 = re.compile(r'^[gG]\d\d?[ _-]?\d\w\w')

# updated to account for local street type conventions as a result of testing sample "G20.osm"
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road",
            "Trail", "Parkway", "Commons", "Crescent", "Gate", "Gardens", "Path", "Terrace", "Way"]

# initial mapping dictionary - this will be updated in data.py
mapping = { "St": "Street",
            "St.": "Street",
            "Rd.": "Road",
            "Ave": "Avenue"
            }

def is_postcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def is_phone_number(elem):
    if "phone" in elem.attrib['k'].lower():
        return (elem.attrib['v'])  

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def audit_number_type(badnumlist, number):
	goodmatch = regex1.match(number)
	if not goodmatch:
		badnumlist.append(number)

def audit_postcode(badpclist, postcode):
    goodmatch2 = regex2.match(postcode)
    if not goodmatch2:
        badpclist.append(postcode)

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    badnumlist = []
    badpclist = []
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                elif is_phone_number(tag):
					audit_number_type(badnumlist, tag.attrib['v'])
                elif is_postcode(tag):
                    audit_postcode(badpclist, tag.attrib['v'])
                
    osm_file.close()
    pprint.pprint("DICTIONARY OF NONSTANDARD STREET TYPES:")
    print "\n"
    pprint.pprint(street_types)
    print "\n"
    pprint.pprint("LIST OF NONSTANDARD FORMAT PHONE NUMBERS:")
    print "\n"
    pprint.pprint(badnumlist)
    print "\n"
    pprint.pprint("LIST OF NONSTANDARD FORMAT POSTCODES:")
    print "\n"
    pprint.pprint(badpclist)

	#pprint.pprint(street_types)
    #pprint.pprint(badnumlist)
    #pprint.pprint(badpclist)
audit(osmfile)

'DICTIONARY OF NONSTANDARD STREET TYPES:'


defaultdict(<type 'set'>, {'Glen': set(['Peel Glen']), 'Rigg': set(['Cander Rigg']), 'West': set(['Paisley Road West']), 'Candleriggs': set(['Candleriggs']), 'Cross': set(['Bridgeton Cross']), 'Hutton': set(['Hutton']), 'street': set(['Brand street', 'Ross street', 'Hope street', 'southbank street', 'shaw street', 'West nile street']), 'Toll': set(['The Toll']), 'East': set(['Drumry Road East']), 'Dovehill': set(['Great Dovehill']), 'Quay': set(['Carrick Quay', 'Pacific Quay']), 'Gallowgate': set(['Gallowgate']), 'Road,': set(['Cumbernauld Road,']), 'Park': set(['Claythorn Park', 'Hanson Park', 'Montfort Park']), 'Holdings': set(['Langbank Holdings']), 'Strret': set(['Garfield Strret']), 'Roystonhill': set(['Roystonhill']), 'Bridgegate': set(['Bridgegate']), 'Drygate': set(['Drygate']), 'Westercraigs': set(['Westercraigs']), 'Murchison': set(['Murchison']), 'Grove': set(['East Nerston Grove', 'Murray Grove']), 'Sreet': set(['Springbank Sreet'

### Street Types

At first glance, based on the number of returns it looks like the initial expected list was not sufficiently comprehensive - however many of these results are not "Wrong" per se, so much as "reflective of the rich linguistic heritage of scotland". These results do not require cleaning and can be left in the data. However there are also some problems with spelling and consistency (Sreet, Strret etc) - these are errors that can be fixed by adding additional terms to our mapping dictionary in data.py using it to swap in something legible wherever one of these issues is encountered.


### Phone Numbers

The returned numbers fall into several categories.

1. There are various permutations of no spaces, spaces and dashes used at different points in each number.
2. Some numbers include an international dialling code. Others omit this and are in local format.
3. Some have both the international dialing code and the local code fully included within the number 
4. Some phone numbers include brackets or spaces between the international and local codes.
5. Some numbers include multiple separate phone numbers.
6. Some numbers just have the wrong number of numbers.

To ensure consistency I decided to impose the format (+44-nnn-nnn-nnnn). +44 is the international dialing code for the uk and replaces the first 0 in the local number.

To address these points, my function in data.py will take each value and remove all non digits. This removes a lot of the degrees of entropy when dealing with the above issues. From there, we can check the length of the string of the  number and use .startswith in order to match each of the various circumstances identified above. Following this, the number is modified to match my specific format.


### Postcodes

Not much to report here - other than the obvious observation that there a few PA2 postcodes in there. These correspond to addresses in renfrewshire. It was in fact myself who inadvertently caught a section of renfrewshire in the map when selecting a square map area that both met the required filesize and covered the glasgow metropolitan area. These won't impair anyone's ability to use the dataset provided they are aware of how it was selected, so I will leave them in.

With regards to the other results - several results are returning only the high level "Area" section of the postcode, IE only the first 2 or 3 characters. To quote wikipedia:

"The structure of a postcode is a one or two-letter postcode area code named for a local town or area of London, one or two digits signifying a district in that region, a space, and then an arbitrary code of one number and two letters. For example, the postcode of the University of Roehampton in London is SW15 5PU, where SW stands for south-west London. The postcode of GCHQ is GL51 0EX, where GL signifies the postal town of Gloucester."

In this case as the area of interest is Glasgow, the postcode area code is G. The district code can be one or two digits, and the arbitrary code (after the space or dash) giving the exact delivery address is denoted by Number Letter Letter. So overall some of our returns are only showing the District code - unfortunately there is no way to fix this. I will not remove these however as they still give some indication of where the node is located, which is better than nothing.

Note - I also intend to create Area codes for nodes that already have postcodes. This is because it allows us to search by district, to get a sense of where all the good amenities are located for example.

My code will also modify all existing postcode consisting of 2 or 3 characters to have "---" appended to them so that it is clear that they are incomplete.




# 3 - Cleaning the data / Exporting to CSV / Creating Database

## Data.py The following code was used to complete these steps

In [ ]:
OSM_PATH = "G129.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

        if element.tag == 'node':
        for attribute in NODE_FIELDS:
            #if key exists
            if element.attrib.get(attribute):
                node_attribs[attribute] = element.attrib[attribute]
            else:
                #insert placeholder value for missing data
                node_attribs[attribute] = "999999999999"
        for child in element:
            nodes = {}
            #create a new dictionary that can be updated with the new tag Areacode without overwriting the postcode
            nodes1 = {}
            problem = PROBLEMCHARS.match(child.attrib['k'])
            lowco = LOWER_COLON.match(child.attrib['k'])
            if problem:
                continue
            elif "phone" in child.attrib['k'].lower():
                nodes['type'] = 'regular'
                nodes['key'] = child.attrib['k']
                nodes['id'] = element.attrib['id']
                if update_phone(child.attrib['v']):
                    nodes["value"] = update_phone(child.attrib['v'])
                    tags.append(nodes) 
            elif lowco:
                nodes['type'] = child.attrib['k'].split(':', 1)[0]
                nodes['key'] = child.attrib['k'].split(':', 1)[1]
                nodes['id'] = element.attrib['id']
                if child.attrib['k'] == "addr:street":
                    nodes["value"] = update_name(child.attrib['v'], mapping)
                    tags.append(nodes)
                elif child.attrib['k'] == "addr:postcode":
                    nodes["value"] = update_postcode(child.attrib['v'])
                    tags.append(nodes)
                    if make_areacode_out_of_postcode(child.attrib['v']):
                        nodes1['type'] = 'addr'
                        nodes1['key'] = 'Areacode'
                        nodes1['id'] = element.attrib['id']
                        nodes1["value"] = make_areacode_out_of_postcode(child.attrib['v']).strip()
                        tags.append(nodes1)
                else:
                    nodes['value'] = child.attrib['v']
                    tags.append(nodes)

            elif "postal_code" in child.attrib['k'].lower():
                nodes['type'] = 'regular'
                nodes['key'] = child.attrib['k']
                nodes['id'] = element.attrib['id']
                nodes["value"] = update_postcode(child.attrib['v'])
                tags.append(nodes)
                if make_areacode_out_of_postcode(child.attrib['v']):
                    nodes1['type'] = 'addr'
                    nodes1['key'] = 'Areacode'
                    nodes1['id'] = element.attrib['id']
                    nodes1["value"] = make_areacode_out_of_postcode(child.attrib['v']).strip()
                    tags.append(nodes1)

            else:
                nodes['type'] = 'regular'
                nodes['key'] = child.attrib['k']
                nodes['id'] = element.attrib['id']
                nodes['value'] = child.attrib['v']
                tags.append(nodes)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for attribute in WAY_FIELDS:
            #if key exists
            if element.attrib.get(attribute):
                way_attribs[attribute] = element.attrib[attribute]
            else:
                #insert placeholder value for missing data
                node_attribs[attribute] = "999999999999"
        #initialise counter to prevent bug occuring if position is reset to zero within the loop
        position = 0
        for child in element:
            wayn = {}
            wayt = {}
            if child.tag == 'nd':
                #create the entry in the dictionary wayn
                if element.attrib['id'] not in way_nodes:
                    wayn['position'] = position
                    wayn['id'] = element.attrib['id']
                    wayn['node_id'] = child.attrib['ref']
                    #we are adding the whole dictionary just created into the way_nodes list using append
                    way_nodes.append(wayn)
                    position += 1
                #however if there is already an associated node for this way, increment position and add as normal
                else:
                    wayn['position'] = position
                    wayn['id'] = element.attrib['id']
                    wayn['node_id'] = child.attrib['ref']
                    way_nodes.append(wayn)
                    position += 1
            elif child.tag == 'tag':
                problem = PROBLEMCHARS.match(child.attrib['k'])
                lowco = LOWER_COLON.match(child.attrib['k'])
                if problem:
                    continue
                elif lowco:
                    wayt['type'] = child.attrib['k'].split(':', 1) [0]
                    wayt['key'] = child.attrib['k'].split(':', 1) [1]
                    wayt['id'] = element.attrib['id']
                    wayt['value'] = child.attrib['v']
                    tags.append(wayt)
                else:
                    wayt['type'] = 'regular'
                    wayt['key'] = child.attrib['k']
                    wayt['id'] = element.attrib['id']
                    wayt['value'] = child.attrib['v']
                    tags.append(wayt)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)

        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

def update_postcode(postcode):
    #strips out all non alphanumeric
    x=re.sub(r'\W+',"", postcode)
    if len(x) == 3:
        postcode = x + "---"
#adds dashes to show that the full postcode is incomplete
    return postcode

def make_areacode_out_of_postcode(postcode):
    #create matchobject giving first 2 or 3 characters
    m = regex3.match(postcode)
    #use group method to convert matchobject into a string
    areacode = m.group(0)
    return areacode


def update_phone(number):
    goodmatch = regex1.match(number)
    if not goodmatch:
        #strips out all non digit values
        x=re.sub(r'\D+',"", number)
        #handle numbers with 44 at the start
        if len(x) == 12 and x.startswith("44"):
            #insert punctuation
            number = "+" + x[0:2] + "-" + x[2:4] + "-" + x[5:7] + "-" + x[8:11]
        #numbers in the 0141 format
        elif len(x) == 11 and x.startswith("0"):
        #drop the 0
            number = "+44" + "-" + x[1:3] + "-" + x[4:6] + "-" + x[7:10]            
        #numbers including both 00 and 44
        elif len(x) == 14 and x.startswith("0044"):
            y = str(list(x)[2:])
            number = "+" + y[0:2] + "-" + y[2:4] + "-" + y[5:7] + "-" + y[8:11]

        #numbers including 0 and 44
        elif len(x) == 13 and x.startswith("044"):
            y = str(list(x)[1:])
            number = "+" + y[0:2] + "-" + y[2:4] + "-" + y[5:7] + "-" + y[8:11]

        #as above but in reverse order
        elif len(x) == 13 and x.startswith("440"):
            del list(x)[2]
            number = "+" + x[0:2] + "-" + x[2:4] + "-" + x[5:7] + "-" + x[8:11]
            
        elif len(x) > 21:
            #note this as potential future improvement - write function to split out the second number and add it as a
            #secondary phone tag
            return

    return number

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping.keys():
            #print 'Before: ' , name
            name = re.sub(m.group(), mapping[m.group()], name)
            #print 'After: ', name
    return name

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road",
            "Trail", "Parkway", "Commons", "Crescent", "Gate", "Gardens", "Path", "Terrace", "Way"]

mapping = { "St": "Street",
            "St.": "Street",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Rd": "Road",
            "Road,": "Road",
            "Sreet": "Street",
            "Strret": "Street",
            "street" : "Street",
            "road": "Road"
            }


street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

regex1 = re.compile(r'\+44\-\d{3}\-\d{3}\-\d{4}')

regex2 = re.compile(r'^[gG]\d\d?[ _-]?\d\w\w')

regex3 = re.compile(r'^[gG]\d\d?')



# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)

Problems encountered:

1. There were issues with missing data causing the reader to crash - this necessitated the use of:

```
else:
                #insert placeholder value for missing data
                node_attribs[attribute] = "999999999999"
```

to ensure that the reader did not crash when encountering missing info, and instead returned a placeholder. This is designed to be easily identifiable.

2. My attempts to create a new element called Areacode based on the first three letters of any postcode were stymied because the values for postcode were being overwritten. This was solved with assistance from the forums and the use of a new dictionary , nodes1, that allowed the new element to be created in the csv for each node with a postcode.

```
elif child.attrib['k'] == "addr:postcode":
                    nodes["value"] = update_postcode(child.attrib['v'])
                    tags.append(nodes)
                    if make_areacode_out_of_postcode(child.attrib['v']):
                        nodes1['type'] = 'addr'
                        nodes1['key'] = 'Areacode'
                        nodes1['id'] = element.attrib['id']
                        nodes1["value"] = make_areacode_out_of_postcode(child.attrib['v']).strip()
                        tags.append(nodes1)                    
```


### Importing CSV to database with required schema

In [ ]:
sqlite_file = "C:\Users\James2SxyBoogaloo\Google Drive\Data\P3 -  Data Wrangling\database.db"
#connect to database
conn =sqlite3.connect(sqlite_file)

cur= conn.cursor()

# NODES_TAGS****************************************************************************

cur.execute('''DROP TABLE IF EXISTS nodes_tags''')
conn.commit()

# Create the table, specifying the column names and data types:
cur.execute('''
    CREATE TABLE nodes_tags(id INTEGER, key STRING, value STRING,type STRING)
''')
# commit the changes
conn.commit()

# Read in the csv file as a dictionary, format the
# data as a list of tuples:
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in dr]

# insert the formatted data
cur.executemany("INSERT INTO nodes_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()

conn.close

#Nodes**************************************************************************************************************

cur.execute('''DROP TABLE IF EXISTS nodes''')
conn.commit()


cur.execute('''
    CREATE TABLE nodes(id INTEGER, lat FLOAT, lon FLOAT, user STRING, uid INTEGER, version STRING, changeset INTEGER, timestamp STRING)
''')

conn.commit()

with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['lat'].decode("utf-8"), i['lon'].decode("utf-8"), i['user'].decode("utf-8"), i['uid'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8")) for i in dr]

cur.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
conn.commit()




#WAYS**************************************************************************************************************

cur.execute('''DROP TABLE IF EXISTS ways''')
conn.commit()

cur.execute('''
    CREATE TABLE ways(id INTEGER, user STRING, uid INTEGER, version STRING, changeset INTEGER, timestamp STRING)
''')
conn.commit()

with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['user'].decode("utf-8"), i['uid'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8")) for i in dr]


cur.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)

conn.commit()



conn.close

#WAYS_NODES**************************************************************************************************************

cur.execute('''DROP TABLE IF EXISTS ways_nodes''')
conn.commit()


cur.execute('''
    CREATE TABLE ways_nodes(id INTEGER, node_id INTEGER, position INTEGER)
''')

conn.commit()


with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['node_id'].decode("utf-8"), i['position'].decode("utf-8")) for i in dr]


cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_db)

conn.commit()


conn.close


#WAYS_TAGS**************************************************************************************************************

cur.execute('''DROP TABLE IF EXISTS ways_tags''')
conn.commit()

cur.execute('''
    CREATE TABLE ways_tags(id INTEGER, key STRING, value STRING, type STRING)
''')
conn.commit()


with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in dr]


cur.executemany("INSERT INTO ways_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)

conn.commit()

conn.close




# 4 - SQL analysis

In [15]:
sqlite_file = "C:\Users\James2SxyBoogaloo\Google Drive\Data\P3 -  Data Wrangling\database.db"
#connect to database
conn =sqlite3.connect(sqlite_file)

c = conn.cursor()

## Top ten cuisines

In [16]:
query = '''
SELECT value, count(*) as num \
FROM nodes_tags \
WHERE key ='cuisine' \
GROUP BY value \
ORDER BY num desc \
LIMIT 10;
'''

def cuisine():
    c.execute(query)
    results = c.fetchall()
    print results

cuisine()

[(u'chinese', 70), (u'indian', 66), (u'italian', 49), (u'sandwich', 40), (u'fish_and_chips', 31), (u'pizza', 21), (u'burger', 18), (u'coffee_shop', 18), (u'regional', 16), (u'asian', 10)]


Chinese narrowly edges out Indian as glasgow's preferred cuisine.

## Top ten amenities

In [17]:
query2 = '''
SELECT value, count(*) as num \
FROM nodes_tags \
WHERE key ='amenity' \
GROUP BY value \
ORDER BY num desc \
LIMIT 10;
'''

def amenity():
    c.execute(query2)
    results = c.fetchall()
    print results

amenity()

[(u'bicycle_parking', 556), (u'post_box', 458), (u'fast_food', 307), (u'restaurant', 268), (u'pub', 255), (u'cafe', 229), (u'telephone', 166), (u'atm', 120), (u'bench', 115), (u'recycling', 98)]


Glasgow benefits from a surfeit of bicycle parking.

## Top ten shops

In [18]:
query3 = '''
SELECT value, count(*) as num \
FROM nodes_tags \
WHERE key ='shop' \
GROUP BY value \
ORDER BY num desc \
LIMIT 10;
'''

def shop():
    c.execute(query3)
    results = c.fetchall()
    print results

shop()

[(u'convenience', 231), (u'hairdresser', 119), (u'clothes', 65), (u'supermarket', 65), (u'bookmaker', 54), (u'newsagent', 50), (u'beauty', 40), (u'bakery', 32), (u'yes', 27), (u'butcher', 24)]


As can be seen, the overwhelming majority of shops in glasgow are convenience stores. There is a surprisingly low number of bookmakers - as this is a proxy for deprivation it would be interesting to explore this further, perhaps by areacode. Outside my flat there are 3 bookkeepers within a 60 second walk.

Note that 'Yes' is getting returned as a type of shop - given that the existence of the shop key implies that a shop exists, this is a potential further opportunity for data cleaning.


## Areacodes with the most Bookmakers

In [19]:
query4 = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value= 'bookmaker') i 
    ON nodes_tags.id=i.id
WHERE nodes_tags.key = 'Areacode'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;
'''

def ACB():
    c.execute(query4)
    results = c.fetchall()
    print results

ACB()

[(u'G1', 3), (u'G2', 1), (u'G4', 1)]


Our query returns only 5 results. This is disappointing given the trouble it took to create the Areacode key.

An examination of the CSV indicates that the vast majority of the bookmakers listed in the data do not have an associated postcode: as a result neither do they have an Areacode. - This isn't a cleaning issue however.

In the interest of getting a meaningful output lets investigate restaurants - perhaps these will have fully completed postcodes.

## Areacodes with the most restaurants


In [20]:
query5 = '''
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value= 'restaurant') i 
    ON nodes_tags.id=i.id
WHERE nodes_tags.key = 'Areacode'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;
'''

def AMR():
    c.execute(query5)
    results = c.fetchall()
    print results

AMR()

[(u'G1', 13), (u'G2', 7), (u'G3', 6), (u'G12', 3), (u'G31', 3), (u'G4', 2), (u'G41', 2), (u'G53', 2), (u'G20', 1), (u'G33', 1)]


Once again we are challenged by the incompleteness of the data. Despite the previous count showing a total of no less than 268 restaurants, our top ten Areacodes only account for a small proportion of this total. However assuming that the likelihood of the restuarant not having an associated postcode (and thus Areacode) is unaffected by location, we can still conclude that G1 must have a significant proportion of the total.

## Conclusion


As discussed above, there are a few issues about completeness of the data as it relates to the postcodes being completed for each node. This meant that our ability to fully utilise the newly created Areacode key to assist our SQL exploration was impaired.

I noticed during my exploration that the bus stops in the data get their information directly from the NAPTAN (government public transport) database, and potentially using a similar tool -such as the post offices publically available database - could be used to add in these postcode values. There wouldn't be any danger of overriding the wrong values as the post office have the most accurate postcodes and these do not change even if a restaurant changes ownership for example. By automating this, some of the pitfalls of relying on user entry data can be avoided.

Additional improvements could be made within the existing dataset.

Key - Shop value - "yes" - this could be addressed by writing code that detects whether there is a secondary key confirming what type of shop it is. This key could then be removed, and used as the value for the Shop key, ovewrwriting 'yes'.

Phone numbers - certain nodes have multiple phone numbers - This could be addressed by splitting out the second number and reassigning it to another key. I did not code a solution to this as this was not a common issue in the data and the complexity of code required to make this compatible with the treatment of every other phone number made it unrealistic.

